More experiments with the `ast` module.


Continued from `p2021-0918a_ast.ipynb`.

## Setup

In [1]:
import ast

In [2]:
AST_FILENAME="<ast>"

In [3]:
AST_MODE="eval"

## Example

In [4]:
expression="sum([x+1 for x in y])"

In [5]:
params={"y":[0,1,2]}

## Parsing

In [6]:
tree = ast.parse(expression,mode=AST_MODE)

In [7]:
type(tree)

ast.Expression

## Compile

In [8]:
obj=compile(tree,AST_FILENAME,AST_MODE)

In [9]:
type(obj)

code

## Evaluate

In [10]:
eval(obj,params)

6

## Walk

In [11]:
vars=[]
for node in ast.walk(tree):
  if isinstance(node,ast.Name):
    vars.append(node.id)
vars

['sum', 'x', 'x', 'y']

This is not correct, of course.
Can we fix it?

In [12]:
for node in ast.walk(tree):
  if isinstance(node,ast.Name):
    print(type(node),node.id,type(node.ctx))
  else:
    print(type(node))

<class 'ast.Expression'>
<class 'ast.Call'>
<class 'ast.Name'> sum <class 'ast.Load'>
<class 'ast.ListComp'>
<class 'ast.Load'>
<class 'ast.BinOp'>
<class 'ast.comprehension'>
<class 'ast.Name'> x <class 'ast.Load'>
<class 'ast.Add'>
<class 'ast.Constant'>
<class 'ast.Name'> x <class 'ast.Store'>
<class 'ast.Name'> y <class 'ast.Load'>
<class 'ast.Load'>
<class 'ast.Store'>
<class 'ast.Load'>


In [13]:
vars={}
for node in ast.walk(tree):
  if isinstance(node,ast.Call):
    vars[node.func.id]=False
  elif isinstance(node,ast.Name):
    if node.id not in vars.keys():
      vars[node.id]=True
    if isinstance(node.ctx,ast.Store):
      vars[node.id]=False
deps=[k for k,v in vars.items() if v]

In [14]:
vars

{'sum': False, 'x': False, 'y': True}

In [15]:
deps

['y']

That works here. But are there more complicated expressions that would still fail?